In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import datetime
import io
import sys
import unicodedata
from datetime import timezone

import numpy as np
import pandas as pd
import pybaseball
import requests
import scipy.stats as stats
from bs4 import BeautifulSoup
from pybaseball import statcast
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_squared_error, r2_score


pybaseball.cache.enable()

In [2]:
Train3 = pd.read_csv("~/Desktop/Desktop - Cameron MacBook Pro/Random-Projects/MLB/Data/DailyPitcherModelTrainingData.csv")
Train3 = Train3.drop(Train3.columns[0], axis=1).fillna(0)

In [3]:
Train4 = Train3.drop(columns = ["BatterTeam", "RoadTeam", "HomeTeam"]).groupby(["SP", "p_throws"]).mean().reset_index()
Train4.drop(columns=[col for col in Train4.columns if 'b' in col.lower()], inplace=True)
Train4

,SP,p_throws,Pitches,PA,Outs,Hits,HR,K,RunExp,RA,...,xFIP10,K%5,K%10,Strike%5,CS%5,CS%10,Whiff%5,Whiff%10,CSW%5,CSW%10
0,AJ Smith-Shawver,R,57.400000,11.800000,9.400000,2.400000,1.000000,2.600000,-0.018164,1.800000,...,4.305229,19.697067,19.697067,43.902100,11.546067,11.546067,11.943567,11.943567,23.489633,23.489633
1,Aaron Ashby,L,79.555556,16.777778,13.555556,4.555556,0.777778,5.000000,0.029510,2.944444,...,3.850306,24.025852,23.364586,47.223667,20.503750,20.853849,12.069667,11.600090,32.572306,32.452989
2,Aaron Civale,R,74.547619,16.809524,13.428571,4.119048,0.547619,4.428571,0.003645,1.952381,...,3.586769,23.115079,22.249923,47.144643,18.262944,18.129995,10.607758,10.446779,28.870893,28.577222
3,Aaron Nola,R,84.796875,20.562500,16.593750,4.750000,0.703125,6.125000,0.004048,2.265625,...,2.540906,27.216221,27.402625,50.724271,19.010685,19.067965,13.199534,13.236727,32.209706,32.304000
4,Aaron Sanchez,R,76.300000,17.600000,12.700000,6.300000,0.600000,3.000000,0.030018,3.100000,...,4.053444,13.028567,12.519783,43.791250,17.495017,18.372163,8.726267,7.923711,26.219483,26.294792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,Zach Thompson,R,79.590909,18.227273,14.090909,5.136364,0.863636,3.454545,0.019287,2.772727,...,3.871938,16.421326,16.399142,43.338023,16.324295,16.381666,10.152295,10.227165,26.476318,26.608695
407,Zack Greinke,R,74.769231,17.807692,13.576923,5.230769,0.673077,2.769231,0.010398,2.307692,...,3.774819,13.957641,13.580703,42.808474,18.108340,18.033216,7.866984,7.877397,25.974631,25.909976
408,Zack Littell,R,66.363636,18.181818,14.272727,4.181818,0.454545,3.272727,-0.006959,1.272727,...,3.331386,15.706682,15.407878,45.002364,15.530939,15.421806,10.096561,9.794180,25.628773,25.216665
409,Zack Thompson,L,63.300000,12.500000,9.900000,4.100000,0.800000,2.700000,0.028562,2.400000,...,3.793940,20.982917,23.021174,46.991483,16.144950,16.228337,10.014183,10.327214,26.161783,26.558125


In [5]:
# Assuming Train4 is your DataFrame
# Step 1: Prepare the data
# Select only numeric columns for comparison
numeric_columns = Train4.select_dtypes(include=[np.number]).columns
data_for_similarity = Train4[numeric_columns]

# Step 2: Normalize the data
scaler = StandardScaler()
normalized_data = scaler.fit_transform(data_for_similarity)

# Step 3: Calculate cosine similarity
similarity_matrix = cosine_similarity(normalized_data)

# Function to find similar players and important features
def find_similar_players_and_features(player_name, k=10):
    if player_name not in Train4['SP'].values:
        return "Player not found"
    
    player_index = Train4[Train4['SP'] == player_name].index[0]
    player_similarities = similarity_matrix[player_index]
    
    # Get indices of top k similar players (excluding the player itself)
    similar_indices = player_similarities.argsort()[::-1][1:k+1]
    
    similar_players = []
    for idx in similar_indices:
        name = Train4.iloc[idx]['SP']
        similarity = player_similarities[idx]
        similar_players.append((name, similarity))
    
    # Calculate feature importance
    player_data = normalized_data[player_index]
    feature_importance = []
    for i, feature in enumerate(numeric_columns):
        importance = np.mean([normalized_data[idx][i] * player_similarities[idx] for idx in similar_indices])
        feature_importance.append((feature, importance))
    
    # Sort features by importance
    feature_importance.sort(key=lambda x: abs(x[1]), reverse=True)
    
    return similar_players, feature_importance[:10]  # Return top 10 important features

# Example usage
player_name = "Spencer Strider"
similar_players, important_features = find_similar_players_and_features(player_name, k=10)

print(f"Players similar to {player_name}:")
for name, similarity in similar_players:
    if similarity > 0.95:
        match_level = "High"
    elif similarity > 0.90:
        match_level = "Medium"
    else:
        match_level = "Low"
    print(f"{name}: Similarity score = {similarity:.2f} ({match_level} match)")

print("\nMost important features for similarity:")
for feature, importance in important_features:
    print(f"{feature}: Importance = {importance:.4f}")

Players similar to Spencer Strider:
Jacob deGrom: Similarity score = 0.98 (High match)
Jesus Luzardo: Similarity score = 0.97 (High match)
Carlos Rodon: Similarity score = 0.97 (High match)
Shohei Ohtani: Similarity score = 0.97 (High match)
Hunter Greene: Similarity score = 0.96 (High match)
Shane McClanahan: Similarity score = 0.95 (High match)
Dylan Cease: Similarity score = 0.95 (High match)
Andrew Heaney: Similarity score = 0.95 (Medium match)
Kevin Gausman: Similarity score = 0.95 (Medium match)
Nick Lodolo: Similarity score = 0.95 (Medium match)

Most important features for similarity:
K: Importance = 1.9812
K%10: Importance = 1.8171
Whiff%10: Importance = 1.8032
K%5: Importance = 1.7717
Whiff%5: Importance = 1.7605
K%: Importance = 1.7600
CSW%10: Importance = 1.5029
CSW%5: Importance = 1.4544
Strike%5: Importance = 1.3890
xFIP10: Importance = -1.0539
